# Data Preprocessing

In this notebook we perform loading, preprocessing and saving of processed Bixi bike history data.

In order for the loading to work, all raw Bixi *.csv* files must be stored inside the '*/data/raw/*' directory. Processed data files are saved into the '*/data/*' directory.

Required imports:

In [1]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<div style="border-bottom: 3px solid black; margin-bottom:5px"></div>
<div style="border-bottom: 3px solid black"></div>

## Loading the Time-Series Data

Load the bixi trip history data and combine it into a single Pandas DataFrame.

Assumes that all *.csv* files are inside the *data/raw/* directory relative to the notebook.

**Note**: importing uses string *dtype* for station IDs because there is one station that does not have an integer ID. Alternatively, we could just find the station in the data files and remove it.

In [2]:
# Get all raw data .csv files excluding the station files
filenames = glob.glob('data/raw/OD_*.csv')

# Load each file and merge into a single data_frame
bixi_df = pd.concat(pd.read_csv(f, index_col='start_date', 
                                   parse_dates=['start_date', 'end_date'],
                                   dtype={'start_station_code': str, 'end_station_code': str}) 
                                   for f in sorted(filenames))

In [3]:
# Preview the dataframe
display(bixi_df)

,start_station_code,end_date,end_station_code,duration_sec,is_member
start_date,,,,,
2014-04-15 00:01:00,6209,2014-04-15 00:18:00,6436,1061,1
2014-04-15 00:01:00,6214,2014-04-15 00:11:00,6248,615,1
2014-04-15 00:01:00,6164,2014-04-15 00:18:00,6216,1031,1
2014-04-15 00:01:00,6214,2014-04-15 00:24:00,6082,1382,1
2014-04-15 00:02:00,6149,2014-04-15 00:08:00,6265,347,1
...,...,...,...,...,...
2019-10-31 23:52:45,6901,2019-11-01 00:05:15,6063,749,1
2019-10-31 23:53:59,6218,2019-11-01 00:23:19,6269,1760,1
2019-10-31 23:55:26,7149,2019-11-01 00:11:33,6338,966,1


In [4]:
# Check if there is any missing values in the data
print("Number of missing values per column:") 
display(bixi_df.isna().sum())

Number of missing values per column:


start_station_code    0
end_date              0
end_station_code      0
duration_sec          0
is_member             0
dtype: int64

## Total Daily Trip History Data

Compute and store the total dality trip history data across all stations.

In [5]:
# Create a new DataFrame with only trip timestamps
total_daily_trips = pd.DataFrame(index=bixi_df.index)

# Add the trip_count column
total_daily_trips.insert(0, 'trip_count', 1)
                                 
# Resample to daily frequency
total_daily_trips = total_daily_trips.resample('D').sum()
                                 
# Preview the resampled data
display(total_daily_trips.head())

,trip_count
start_date,
2014-04-15,1577
2014-04-16,3180
2014-04-17,5852
2014-04-18,5325
2014-04-19,6031


In [6]:
# Save the data into a csv file
total_daily_trips.to_csv('data/total_daily_trips.csv')

## Station Daily Trip History Data

Compute and store the daily bike demand (outgoing trip history) and dock demand (incoming trip history) data for a single station.

### Group by station

We can group the data by either start or end station in order to create separate time-series for each station:
* Grouping by start station allows us to estimate overall bike demand using the number of outgoing trips from the station
* Grouping by end station allows us to estimate overall dock demand using the number of incoming trips to the station

In [7]:
# Group the data by start and end station
bike_demand_gb = bixi_df.groupby('start_station_code')
dock_demand_gb = bixi_df.groupby('end_station_code')

### Extract and save station data

Define helpers function to extract and save the daily trip data for a single station.

In [8]:
def extract_station_data(station_gb, station_id): 
    # Get the station data
    station_data = station_gb.get_group(station_id)
    
    # Resample the data to daily frequency
    station_data.insert(0, 'trip_count', 1)
    station_data = station_data[['trip_count']]
    station_data = station_data.resample('D').sum()

    return station_data

In [9]:
def save_station_data(station_data, station_name, demand_type):
    if demand_type != 'bike' and demand_type != 'dock':
        print("Error: invalid demand type", demand_type)
        return
    
    filename = f'data/{station_name}_{demand_type}_demand.csv'
    station_data.to_csv(filename)

### Find most popular station

In [10]:
# Print stations sorted according to the total number of trips
print("Bike demand group sizes:")
display(bike_demand_gb.size().sort_values(ascending=False).head())

print("Dock demand group sizes:")
display(dock_demand_gb.size().sort_values(ascending=False).head())

Bike demand group sizes:


start_station_code
6184    259233
6100    249178
6064    206396
6136    199872
6221    187453
dtype: int64

Dock demand group sizes:


end_station_code
6015    269810
6100    249367
6012    248450
6184    243197
6064    211985
dtype: int64

The station with ID *6184* is most popular for bike demand and the 3rd most popular for dock demand. Let's find what it is.

In [11]:
# Load most recent station data
stations_df = pd.read_csv('data/raw/stations.csv', index_col='code')

# Print most popular station data
display(stations_df.loc[6184])

name         Métro Mont-Royal (Rivard / du Mont-Royal)
latitude                                     45.524673
longitude                                    -73.58255
Name: 6184, dtype: object

The overall most popular station is Métro Mont-Royal. We will use it as our station of choice.

### Extract most popular station data

In [12]:
# Get Metro Mont-Royal data
mr_bike_demand = extract_station_data(bike_demand_gb, '6184')
mr_dock_demand = extract_station_data(dock_demand_gb, '6184')

In [13]:
# Preview the station data
display(mr_bike_demand.head())
display(mr_dock_demand.head())

,trip_count
start_date,
2014-04-15,31
2014-04-16,67
2014-04-17,140
2014-04-18,108
2014-04-19,128


,trip_count
start_date,
2014-04-15,39
2014-04-16,82
2014-04-17,136
2014-04-18,120
2014-04-19,118


In [14]:
# Save Mont Royal data
save_station_data(mr_bike_demand, 'mr', 'bike')
save_station_data(mr_dock_demand, 'mr', 'dock')

### Extract and save any station data

Use this cell to extract and save data for any station.

In [15]:
RUN_CELL = False

if RUN_CELL:
    station_id   = '6015'   # Put your station ID here
    station_name = 'berri'  # Put the station name here

    # Extract station data
    bike_demand = extract_station_data(bike_demand_gb, station_id)
    dock_demand = extract_station_data(dock_demand_gb, station_id)

    # Save station data
    save_station_data(bike_demand, station_name, 'bike')
    save_station_data(dock_demand, station_name, 'dock')

Use the next cell to extract and save station data for every station that has data in our period of interest.

In [16]:
RUN_CELL = False

if RUN_CELL:
    # Get station ids
    station_ids = bike_demand_gb.groups.keys()

    # For each station id, extract data and check if it has values for every day
    # between 2014 and 2019
    for station_id in station_ids:
        station_data = extract_station_data(bike_demand_gb, station_id)['2014':'2019']        
        if(len(station_data) == 2087):
            # Save data for passing stations
            save_station_data(station_data, station_id, 'bike')

## Weather Data Processing

Here we will load, process and save weather data as a parallel time-series to our Time-Series data.

First we define a helper function to load and process weather data for a single station.

Raw weather *.csv* data for each station must be located in the *data/weather/{station_name}/* directory in order to be loaded.

In [17]:
def load_and_process(station_name):
    # Get all raw weather data .csv files
    filenames = glob.glob(f'data/weather/{station_name}/*.csv')

    # Define the list of useful columns to extract
    cols=['Date/Time', 'Max Temp (°C)', 'Min Temp (°C)', 'Mean Temp (°C)', 'Total Precip (mm)', 'Spd of Max Gust (km/h)']

    # Load each file and merge into a single data_frame
    weather_df = pd.concat(pd.read_csv(f, index_col='Date/Time',
                                   usecols = cols,
                                   parse_dates=['Date/Time']) 
                                   for f in sorted(filenames))
    
    # If wind speed in denoted as <31, replace it with 31 and convert the column to float 
    weather_df.loc[weather_df['Spd of Max Gust (km/h)'] == '<31', 'Spd of Max Gust (km/h)'] = 31
    weather_df = weather_df.astype({'Spd of Max Gust (km/h)': 'float64'})
    
    return weather_df

As our main station we use the *McTavish* weather station, located in the center of Montreal.

As we can see, there are quite a few missing values in the station's data that have to be filled.

In [18]:
weather_df = load_and_process('mctavish')
display(weather_df)

print("Number of null values per column:")
display(weather_df.isnull().sum())

,Max Temp (°C),Min Temp (°C),Mean Temp (°C),Total Precip (mm),Spd of Max Gust (km/h)
Date/Time,,,,,
2014-01-01,-15.2,-23.9,-19.6,0.0,31.0
2014-01-02,-22.3,-27.4,-24.9,0.0,37.0
2014-01-03,-19.0,-23.6,-21.3,1.0,31.0
2014-01-04,-6.6,-21.3,-14.0,0.0,31.0
2014-01-05,0.2,-10.6,-5.2,8.0,31.0
...,...,...,...,...,...
2019-12-27,3.5,-4.8,-0.6,7.2,NaN
2019-12-28,4.2,-0.2,2.0,0.2,NaN
2019-12-29,0.8,-2.9,-1.1,0.0,31.0


Number of null values per column:


Max Temp (°C)              71
Min Temp (°C)              43
Mean Temp (°C)             73
Total Precip (mm)         278
Spd of Max Gust (km/h)    435
dtype: int64

### Using another station data in close proximity

In order to fill the missing station values, we will use data from two supplementary stations located in close proximity (within 10km) to the McTavish station.

In [19]:
# Load St. Hubert weather data
weather_df1 = load_and_process('intl_a')
print("Number of null values per column:")
display(weather_df1.isnull().sum())

Number of null values per column:


Max Temp (°C)               8
Min Temp (°C)               4
Mean Temp (°C)              8
Total Precip (mm)          19
Spd of Max Gust (km/h)    106
dtype: int64

In [20]:
# Load Int. Airport weather data
weather_df2 = load_and_process('st_hubert')
print("Number of null values per column:")
display(weather_df2.isnull().sum())

Number of null values per column:


Max Temp (°C)             202
Min Temp (°C)             192
Mean Temp (°C)            203
Total Precip (mm)         223
Spd of Max Gust (km/h)    293
dtype: int64

In [21]:
# Fill missing values using the supplementaty weather data
weather_df = weather_df.fillna(weather_df1)
weather_df = weather_df.fillna(weather_df2)
weather_df.isnull().sum()

Max Temp (°C)              0
Min Temp (°C)              0
Mean Temp (°C)             0
Total Precip (mm)          0
Spd of Max Gust (km/h)    77
dtype: int64

As we are still have a few missing values, we will use interpolation to fill them in.

In [22]:
weather_df.interpolate(axis=1, inplace=True)
weather_df.isnull().sum()

Max Temp (°C)             0
Min Temp (°C)             0
Mean Temp (°C)            0
Total Precip (mm)         0
Spd of Max Gust (km/h)    0
dtype: int64

Now that all missing values are filled in, let's take a quick look at the general features of the weather dataset.

In [23]:
weather_df.describe()

,Max Temp (°C),Min Temp (°C),Mean Temp (°C),Total Precip (mm),Spd of Max Gust (km/h)
count,2191.000000,2191.000000,2191.000000,2191.000000,2191.000000
mean,11.904747,3.787951,7.842492,3.142492,33.042355
std,12.560176,11.740618,12.061086,7.169566,8.939278
min,-22.300000,-27.800000,-24.900000,0.000000,0.000000
25%,1.800000,-4.800000,-1.500000,0.000000,31.000000
50%,12.300000,4.800000,8.400000,0.000000,31.000000
75%,23.500000,14.200000,18.800000,2.600000,35.000000
max,36.600000,25.000000,30.000000,77.800000,115.000000


### Save weather data

Finally, we can save the processed weather data to a *.csv* file for later use.

In [24]:
filename = f'data/weather.csv'
weather_df.to_csv(filename)